In [8]:
import mmcv
print(mmcv.__version__)

# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

import torch
print(torch.cuda.is_available())
print(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

1.6.2
True
cuda


In [9]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/retinanet/retinanet_r50_fpn_1x_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.ann_file = root + 'skfold/cv_train_skfold1.json'
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
#cfg.data.val.ann_file = root + 'val.json' # val json 정보
cfg.data.val.ann_file = root + 'skfold/cv_val_skfold1.json'
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/retinanet_r50_fpn_1x_trash_skf1'

cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [10]:
cfg.keys()
cfg.data.keys()

dict_keys(['samples_per_gpu', 'workers_per_gpu', 'train', 'val', 'test'])

In [11]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [12]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [19]:
print(cfg.model.backbone)

import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)

import torchvision.models as models

# ResNet50 모델 불러오기
resnet50 = models.resnet50(pretrained=True)

# 모델의 구조를 출력하여 모델이 제대로 로드되었는지 확인
print(resnet50)

{'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}
PyTorch version: 1.12.1
Torchvision version: 0.13.1
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [20]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-16 15:45:09,771 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-01-16 15:45:09,773 - mmcv - INFO - load model from: torchvision://resnet50
2024-01-16 15:45:09,774 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-01-16 15:45:09,863 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-01-16 15:45:09,891 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-16 15:45:09,948 - mmcv - INFO - initialize RetinaHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'retina_cls', 'std': 0.01, 'bias_prob': 0.01}}
2024-01-16 15:45:10,001 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from torchvision://resnet50 
 
2024-01-16 15:45:10,002 - mmcv - INFO

In [21]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-01-16 15:45:20,157 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-16 15:45:20,159 - mmdet - INFO - Start running, host: root@instance-5252, work_dir: /data/ephemeral/home/baseline/mmdetection/work_dirs/retinanet_r50_fpn_1x_trash2
2024-01-16 15:45:20,160 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook               

AssertionError: The `num_classes` (11) in RetinaHead of MMDataParallel does not matches the length of `CLASSES` 10) in CocoDataset